# KNN 회귀

In [ ]:
import os
import pandas as pd
import numpy as np
import hds
from plt_rcs import *

In [ ]:
plt.rc(group='figure', figsize=(4,4))

In [ ]:
os.getcwd()

In [ ]:
os.chdir('../../data')

In [ ]:
sorted(os.listdir())

In [ ]:
objs = pd.read_pickle('Diabetes.pkl')

In [ ]:
globals().update(objs)

In [ ]:
%whos

In [ ]:
X_train, X_valid, y_train, y_valid = X_train, X_valid, y_train, y_valid

## 데이터 표준화

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train = scaler.fit_transform(X=X_train)
X_valid = scaler.transform(X=X_valid)

In [ ]:
pd.DataFrame(data=X_valid).describe().round(3)

## 가중치 없는 KNN 회귀 모델 학습

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
model_unif = KNeighborsRegressor(p=1)

In [ ]:
model_unif.fit(X=X_train, y=y_train)

In [ ]:
model_unif.score(X=X_train, y=y_train)
# 0.5783751948120422
model_unif.score(X=X_valid, y=y_valid)
# 0.34874094754106255

In [ ]:
distances, indices = model_unif.kneighbors(X=X_valid)

In [ ]:
distances[0]

In [ ]:
kth_distance = pd.Series(data=distances[:, -1])
kth_distance

In [ ]:
kth_distance.describe().round(3)

In [ ]:
kth_distance.sort_values().tail()

In [ ]:
sns.histplot(x=kth_distance, binrange=(1, 7), binwidth=0.5)
plt.show()

## 가중치 있는 KNN 회귀 모델

In [ ]:
model_dist = KNeighborsRegressor(weights='distance', p=1)

In [ ]:
model_dist.fit(X=X_train, y=y_train)

In [ ]:
model_dist.score(X=X_train, y=y_train)
# 1.0
model_dist.score(X=X_valid, y=y_valid)
# 0.35088449132245114

In [ ]:
y_pred_unif = model_unif.predict(X=X_valid)
y_pred_dist = model_dist.predict(X=X_valid)

In [ ]:
hds.stat.regmetrics(y_true=y_valid, y_pred=y_pred_unif)

In [ ]:
hds.stat.regmetrics(y_true=y_valid, y_pred=y_pred_dist)

## 최적의 이웃 개수 탐색

In [ ]:
def valid_score(k):
    model = KNeighborsRegressor(n_neighbors=k, weights='distance', p=1)
    model.fit(X_train, y_train)
    score = model.score(X=X_valid, y=y_valid)
    return score

In [ ]:
ks = range(1, 100)

In [ ]:
vl_rsq = [valid_score(k) for k in ks]

In [ ]:
np.max(vl_rsq)
# np.float64(0.36093215506286147)

In [ ]:
index = np.argmax(vl_rsq)

In [ ]:
best_k = ks[index]
best_k

In [ ]:
sns.lineplot(x=ks, y=vl_rsq)
plt.show()

In [ ]:
model_best = KNeighborsRegressor(n_neighbors=best_k, p=1, weights='distance')
model_best.fit(X=X_train, y=y_train)

In [ ]:
model_best.score(X=X_train, y=y_train)
# 1.0
model_best.score(X=X_valid, y=y_valid)
# 0.36093215506286147

In [ ]:
y_pred_best = model_best.predict(X_valid)

In [ ]:
hds.stat.regmetrics(y_true=y_valid, y_pred=y_pred_best)